# Environment test for WatsonX.ai  

In [1]:
import os
from dotenv import load_dotenv
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes
from langchain.vectorstores import FAISS
from langchain.embeddings import TensorflowHubEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
parameters = {
    GenParams.DECODING_METHOD: "greedy",
    GenParams.MAX_NEW_TOKENS: 200,
    GenParams.MIN_NEW_TOKENS: 0,
    GenParams.STOP_SEQUENCES: ["\n"],
    GenParams.REPETITION_PENALTY:1
    }


load_dotenv()
project_id = os.getenv("PROJECT_ID", None)
credentials = {
        "url": "https://us-south.ml.cloud.ibm.com",
        "apikey": os.getenv("API_KEY", None)
        }    
#this cell should never fail, and will produce no output
import requests

def getBearer(apikey):
    form = {'apikey': apikey, 'grant_type': "urn:ibm:params:oauth:grant-type:apikey"}
    print("About to create bearer")
#    print(form)
    response = requests.post("https://iam.cloud.ibm.com/oidc/token", data = form)
    if response.status_code != 200:
        print("Bad response code retrieving token")
        raise Exception("Failed to get token, invalid status")
    json = response.json()
    if not json:
        print("Invalid/no JSON retrieving token")
        raise Exception("Failed to get token, invalid response")
    print("Bearer retrieved")
    return json.get("access_token")

credentials["token"] = getBearer(credentials["apikey"])
from ibm_watson_machine_learning.foundation_models import Model
model_id = ModelTypes.LLAMA_2_70B_CHAT

# Initialize the Watsonx foundation model
llama_model = Model(
    model_id=model_id, 
    params=parameters, 
    credentials=credentials,
    project_id=project_id)


# Function to get text from PDF documents
def get_pdf_text(pdf_docs):
    text = ""
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)
        text += " ".join(page.extract_text() for page in pdf_reader.pages)
    return text

# Function to split text into chunks
def get_text_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks

# Function to create a vector store
def get_vectorstore(text_chunks):
    url = "https://tfhub.dev/google/universal-sentence-encoder-multilingual/3"
    #embeddings = OpenAIEmbeddings()
    #embeddings = HuggingFaceInstructEmbeddings() 
    embeddings  = TensorflowHubEmbeddings(model_url=url)   
    vectorstore = FAISS.from_texts(texts=text_chunks, embedding=embeddings)
    return vectorstore

# Function to create a conversation chain
def get_conversation_chain(vectorstore):
    #llm = ChatOpenAI()
    #llm = HuggingFaceHub(repo_id="google/flan-t5-xxl", model_kwargs={"temperature":0.5, "max_length":512})
    llm=llama_model.to_langchain()
    memory = ConversationBufferMemory(
        memory_key='chat_history', return_messages=True)
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(),
        memory=memory
    )
    return conversation_chain

About to create bearer
Bearer retrieved


In [2]:
# get the current working directory
current_working_directory = os.getcwd()# Path
# Join various path components 
pdf_path=os.path.join(current_working_directory, "documents", "example.pdf")

In [3]:
# Get PDF text and split into chunks
raw_text = get_pdf_text([pdf_path])

In [4]:
text_chunks = get_text_chunks(raw_text)

In [5]:
# Create vector store and conversation chain
vectorstore = get_vectorstore(text_chunks)

In [6]:
llm=llama_model.to_langchain()
memory = ConversationBufferMemory(memory_key='chat_history',
                                    return_messages=True)

In [7]:
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    memory=memory)

In [45]:
query = "What is the topic about"

In [46]:
prompt={"question": query}

In [47]:
result = conversation_chain(prompt)

In [48]:
result

{'question': 'What is the topic about',
 'chat_history': [HumanMessage(content='Instruction: Answer the following question using only information from the article. Question: What is the topic about', additional_kwargs={}, example=False),
  AIMessage(content=" The article discusses the Transformer model, a new neural network architecture for natural language processing tasks. It describes the model's attention mechanism, which allows it to jointly attend to information from different representation subspaces at different positions. The article also discusses the training regime for the model, including the training data and batching.", additional_kwargs={}, example=False),
  HumanMessage(content='Instruction: Answer the following question using only information from the article.If there is no good answer in the article, say "I don\'t know". Question: What is the topic about', additional_kwargs={}, example=False),
  AIMessage(content='  The Transformer model is primarily used for sequenc


To include instructions to the large language model in the python code above, you can use the **parameters** argument when calling the conversation_chain.generate() method. This argument is a dictionary that can contain any additional parameters that you want to pass to the model.

In [33]:
parameters = {"instruction": "Answer the following question using only information from the article. If there is no good answer in the article, say I don't know"}

In [35]:
query = "What is the topic about"

In [36]:
prompt={"question": query}

In [40]:
#help(conversation_chain)

In [38]:
result = conversation_chain(prompt, parameters)


In [39]:
result

{'answer': '  The article discusses the Transformer model, a new architecture for neural machine translation that relies entirely on self-attention mechanisms to compute representations of its input and output.'}

In [41]:
query = "What is the temperature in Genova"
prompt={"question": query}

In [42]:
result = conversation_chain(prompt,parameters)

In [43]:
result

{'answer': "  I don't know.\n"}

In [44]:
memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='Instruction: Answer the following question using only information from the article. Question: What is the topic about', additional_kwargs={}, example=False), AIMessage(content=" The article discusses the Transformer model, a new neural network architecture for natural language processing tasks. It describes the model's attention mechanism, which allows it to jointly attend to information from different representation subspaces at different positions. The article also discusses the training regime for the model, including the training data and batching.", additional_kwargs={}, example=False), HumanMessage(content='Instruction: Answer the following question using only information from the article.If there is no good answer in the article, say "I don\'t know". Question: What is the topic about', additional_kwargs={}, example=False), AIMessage(content='  The Transformer model is primarily used for seque